In [7]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook')

In [8]:
img_metadata = pd.read_csv("../dataset/Data_Entry_2017.csv")
unusable_images = pd.read_csv("../dataset/cxr14_bad_labels.csv").drop(["Index"], axis=1)

In [9]:
finding_labels = set()
for labels in pd.unique(img_metadata["Finding Labels"]):
    for label in labels.split("|"):
        finding_labels.add(label.lower().replace(" ", "_"))

finding_labels = list(finding_labels)
finding_labels.sort()
print("All labels:")
for i, label in enumerate(finding_labels):
    print(f"{i + 1}. {label}")
print(f"Total labels: {len(finding_labels)}")

All labels:
1. atelectasis
2. cardiomegaly
3. consolidation
4. edema
5. effusion
6. emphysema
7. fibrosis
8. hernia
9. infiltration
10. mass
11. no_finding
12. nodule
13. pleural_thickening
14. pneumonia
15. pneumothorax
Total labels: 15


In [10]:
dx_series = pd.Series(list(finding_labels))
dx_series.to_csv("../dataset/dx_labels.csv", index=False, header=["dx_labels"])

In [11]:
img_metadata.rename(columns = {"Image Index": "img_filename", "Patient ID": "pt_id", "Patient Age": "pt_age", "Patient Gender": "pt_sex", "View Position": "view_position", "Image Width": "img_width", "Image Height": "img_height", "Spacing X": "x_spacing", "Spacing Y": "y_spacing"}, inplace=True)
img_metadata.columns

Index(['img_filename', 'Finding Labels', 'Follow-up #', 'pt_id', 'pt_age',
       'pt_sex', 'view_position', 'img_width', 'img_height', 'x_spacing',
       'y_spacing'],
      dtype='object')

In [12]:
def has_label(label, row):
    return 1 if label in row["Finding Labels"].lower().replace(" ", "_").split("|") else 0

for label in finding_labels:
    img_metadata[label] = img_metadata.apply(lambda row: has_label(label, row), axis=1)

img_metadata = img_metadata.drop(["Finding Labels", "Follow-up #"], axis=1)

In [13]:
unusable_images.head()

,File label,Inverted,Not frontal,Rotated
0,00000583_024.png,1.0,0.0,0.0
1,00002180_000.png,1.0,0.0,0.0
2,00002300_026.png,1.0,0.0,0.0
3,00002371_015.png,1.0,0.0,0.0
4,00006209_001.png,1.0,0.0,0.0


In [14]:
to_drop = []
unusable_list = unusable_images["File label"].tolist()
for index, row in img_metadata.iterrows():
    if row["img_filename"] in unusable_list:
        to_drop.append(index)
print(f"Total images to drop: {len(pd.unique(unusable_images['File label']))}")
print(f"Total indexes found: {len(to_drop)}")


Total images to drop: 257
Total indexes found: 257


In [15]:
usable_imgs = img_metadata.drop(to_drop, axis=0)
print(f"Total images before drop: {len(img_metadata)}")
print(f"Images usable: {len(usable_imgs)}")
print(f"Total dropped: {len(img_metadata) - len(usable_imgs)}")

Total images before drop: 112120
Images usable: 111863
Total dropped: 257


In [16]:
usable_imgs = usable_imgs[usable_imgs["pt_age"] < 100]
usable_imgs.describe()

,pt_id,pt_age,img_width,img_height,x_spacing,y_spacing,atelectasis,cardiomegaly,consolidation,edema,...,emphysema,fibrosis,hernia,infiltration,mass,no_finding,nodule,pleural_thickening,pneumonia,pneumothorax
count,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,...,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000,111847.000000
mean,14352.069854,46.873729,2646.817215,2486.516482,0.155634,0.155634,0.103195,0.024811,0.041664,0.020519,...,0.022450,0.015047,0.002030,0.177618,0.051579,0.538074,0.056488,0.030202,0.012767,0.047315
std,8404.640184,16.593094,340.900387,401.416234,0.016178,0.016178,0.304214,0.155548,0.199821,0.141768,...,0.148143,0.121742,0.045005,0.382192,0.221177,0.498550,0.230862,0.171143,0.112270,0.212312
min,1.000000,1.000000,1143.000000,966.000000,0.115000,0.115000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7317.000000,35.000000,2500.000000,2048.000000,0.143000,0.143000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,13995.000000,49.000000,2520.000000,2544.000000,0.143000,0.143000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,20673.000000,59.000000,2992.000000,2991.000000,0.168000,0.168000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,30805.000000,95.000000,3827.000000,3567.000000,0.198800,0.198800,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
usable_imgs.to_csv("../dataset/usable_img_metadata.csv", index=False)